## Import libraries

In [5]:
#!pip install selenium

    100% |████████████████████████████████| 921kB 1.3MB/s ta 0:00:01


In [1]:
import requests
import bs4
from bs4 import BeautifulSoup
import numpy as np
import json
import pandas as pd
import time

from selenium import webdriver
from selenium.webdriver.common.keys import Keys

# Job search Webcrawler

In [2]:
## sleep
def sleep_func(time_sec):
    print ('wating'),
    interval = 1
    c = 0
    while c < time_sec:
        time.sleep(interval)
        print ('.'),
        c += 1

In [3]:
def dict_append(dictionary, idx_counter, new_key, old_key1, old_key2, old_key3=None):
# add hiring organisation to ad dictionary
    if old_key3 == None:
        dictionary[idx_counter][new_key] = dictionary[idx_counter][old_key1][old_key2]
        print('.'),
        return dictionary
    else:
        if old_key3 in dictionary[idx_counter][old_key1][old_key2].keys():
            dictionary[idx_counter][new_key] = dictionary[idx_counter][old_key1][old_key2][old_key3]
            print('.'),
            return dictionary

In [20]:
def seek_webcrawler(cities, jobs, pages, salary_ranges):
    
    start_time = time.time()
    driver = webdriver.Chrome('/home/vicki/Desktop/chromedriver') 
    
    ads_dict = {}
    idx_counter = 0
    exception = 0
    exception_list = []
    # repeat search for each city
    for city in cities:
        print ('starting city {}'.format(city)),
        # repeat search for each job type
        for job in jobs:
            print ('for {} jobs'.format(job)),
            # iterate search result pages
            for pgs in range(1,pages+1):
                print ('searching page {}\n\n'.format(pgs)),
                for salary in salary_ranges:
                    # make page request: Seek address format: https://www.seek.com/__jobtitle__+'jobs/'in-'+__city-name__'?page=''__#__
                    # https://www.seek.com.au/data-scientist-jobs/in-All-Melbourne-VIC?salaryrange=0-30000&salarytype=annual
                    #salary = '70000-100000'
                    url = ('https://www.seek.com.au/'+str(job)+'-jobs'+'/in-'+str(city)+'?page='+str(pgs)+'salaryrange='+str(salary)+'&salarytype=annual')
                    headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36"}
                    page = requests.get(url=url, headers=headers)
    ###########     
    # https://www.quora.com/Web-Crawling-How-do-websites-tell-if-their-site-is-being-accessed-by-a-crawler-or-an-actual-user

                    print ('{} {}\n'.format(page, url))

                    #parse html information to BS
                    soup = BeautifulSoup(page.text, 'html.parser')
                    print ('.'),

                    # Pull ad data from script tag for each page # add to dictionary
                    for script in soup.find_all(name='script', attrs={'type':'application/ld+json'}):

                        print ('.'),

                        # Get the text of each script tag
                        script_text = script.text
                        # Decode json format
                        ad_data = json.loads(script_text)
                        # add json dict to ads_dict using counter as index
                        if 'legalName' not in ad_data.keys() and 'mainEntityOfPage' not in ad_data.keys(): 
                            ads_dict[idx_counter] = (ad_data)

                            # add hiring organisation to ad dictionary
                            dict_append(ads_dict, idx_counter, 'employer', 'hiringOrganization', 'name') 

                            # add city to ad dictionary
                            dict_append(ads_dict, idx_counter, 'city', 'jobLocation', 'address', 'addressRegion') 

                            # add locality to ad dictionary
                            dict_append(ads_dict, idx_counter, 'locality', 'jobLocation', 'address', 'addressLocality') 

                            # add job title search to dictionary
                            ads_dict[idx_counter]['job_search_key'] = job
                            # add salary search to dictionary 
                            ads_dict[idx_counter]['salary'] = salary

                            # add job description to ad dictionary
                            driver.get(ads_dict[idx_counter]['url'])

                            try:
                                if driver.find_element_by_class_name('templatetext') is None:
                                    print ('no description')
                                else:
                                    description = driver.find_element_by_class_name('templatetext')
                                    ads_dict[idx_counter]['description'] = description.text
                                    print('.'),
                            except:
                                print ('exception @ site ---> {}'.format(ads_dict[idx_counter]['url'])),
                                exception += 1
                                exception_list.append(ads_dict[idx_counter]['url'])

                            if idx_counter !=0:    
                                if idx_counter % 50 == 0:
                                    sleep_func(60)
                                    print ('\n')
                                if idx_counter % 10 == 0:
                                    sleep_func(10)
                                    print ('\n')
                            ## add 1 to counter
                            idx_counter += 1

                            print (' Completed search {} for {} in {}\n'.format(idx_counter, job, city))
                            driver.delete_all_cookies()
                        else:
                            print 'seek info header'

    webdriver.Chrome.close
    print ('\nYour search is complete -- {} results returned'.format(idx_counter))
    if len(exception_list)>0:
        print ('There were {} exeptions @:\n'.format(exception, exception_list))
    print ('\nElapsed time: {} seconds'.format(time.time()-start_time))
    return (ads_dict)

## Web crawler conditions

In [21]:
## Set scrapping conditions 

cities = ['All-Melbourne-VIC'] #, 'All-Sydney-NSW', 'All-Brisbane-QLD', 'All-Adelaide-SA','All-Perth-WA']
jobs = ['data-scientist'] #, 'data-analyst', 'business-intelligence', 'business-analyst', 'data-architect',
       #'data-engineer', 'statistician', 'research-scientist', 'analyst'] #etc
salary_ranges = ['0-60000','60000-90000','90000-120000', '120000-200000']
# 20 listings per page
pages = 1

## Run Webcrawler

In [22]:
ads_dict = seek_webcrawler(cities, jobs, pages, salary_ranges)

starting city All-Melbourne-VIC for data-scientist jobs searching page 1

<Response [200]> https://www.seek.com.au/data-scientist-jobs/in-All-Melbourne-VIC?page=1salaryrange=0-60000&salarytype=annual

. . seek info header
. seek info header
. . . . .  Completed search 1 for data-scientist in All-Melbourne-VIC

. . . .  Completed search 2 for data-scientist in All-Melbourne-VIC

. . . . .  Completed search 3 for data-scientist in All-Melbourne-VIC

. . . . .  Completed search 4 for data-scientist in All-Melbourne-VIC

. . . . .  Completed search 5 for data-scientist in All-Melbourne-VIC

. . . . .  Completed search 6 for data-scientist in All-Melbourne-VIC

. . . .  Completed search 7 for data-scientist in All-Melbourne-VIC

. . . . .  Completed search 8 for data-scientist in All-Melbourne-VIC

. . . . .  Completed search 9 for data-scientist in All-Melbourne-VIC

. . . . .  Completed search 10 for data-scientist in All-Melbourne-VIC

. . . . wating . . . . . . . . . . 

 Completed sear

In [24]:
ads_dict[0]

{u'@context': u'http://schema.org',
 u'@type': u'JobPosting',
 'city': u'Melbourne',
 u'datePosted': u'2017-09-18T14:00:00Z',
 u'description': u"About us\nCrypton Analytics is a high performing team that is passionate about the power of analytics.This is a rare opportunity to work within an intimate team building predictive models for cryptocurrency markets and sport. \nWhat's in it for you?\nModern offices based in the heart of Melbourne CBD\nCompletely flexible hours and working arrangements\nAn approachable work culture that values work-life balance\nEnjoy autonomy in your role as you work through interesting challenges within a brand-new space\nOpportunity for growth and professional development \nAbout the position\nReporting to the Head Data Scientist, you will explore large datasets, discover trends and patterns and extract insights to better assist our clients.\nDaily responsibilities include:\nGather and process raw data at scale (including writing systems and scripts, calling

## Make dataframe

In [25]:
jobs_df = pd.DataFrame(ads_dict)
jobs_df = jobs_df.T

In [31]:
# Too many jobs have a wide salary range

jobs_df[jobs_df.duplicated(subset='description')].sort_values(['employer','salary'])

,@context,@type,city,datePosted,description,employer,employmentType,hiringOrganization,industry,jobLocation,job_search_key,locality,salary,title,url
68,http://schema.org,JobPosting,Melbourne,2017-09-13T23:55:43Z,Aurec is currently looking to engage an experi...,Aurec,Contract/Temp,"{u'@type': u'Organization', u'name': u'Aurec'}",Information & Communication Technology,"{u'@type': u'Place', u'address': {u'addressLoc...",data-scientist,CBD & Inner Suburbs,120000-200000,Data Scientist - Melbourne - Contract,https://www.seek.com.au/job/34401554
26,http://schema.org,JobPosting,Melbourne,2017-09-13T23:55:43Z,Aurec is currently looking to engage an experi...,Aurec,Contract/Temp,"{u'@type': u'Organization', u'name': u'Aurec'}",Information & Communication Technology,"{u'@type': u'Place', u'address': {u'addressLoc...",data-scientist,CBD & Inner Suburbs,60000-90000,Data Scientist - Melbourne - Contract,https://www.seek.com.au/job/34401554
47,http://schema.org,JobPosting,Melbourne,2017-09-13T23:55:43Z,Aurec is currently looking to engage an experi...,Aurec,Contract/Temp,"{u'@type': u'Organization', u'name': u'Aurec'}",Information & Communication Technology,"{u'@type': u'Place', u'address': {u'addressLoc...",data-scientist,CBD & Inner Suburbs,90000-120000,Data Scientist - Melbourne - Contract,https://www.seek.com.au/job/34401554
64,http://schema.org,JobPosting,Melbourne,2017-09-17T16:59:45Z,"Lead Data Scientist, Director Level, Bangkok\n...",Big Wednesday,Full Time,"{u'@type': u'Organization', u'name': u'Big Wed...",Information & Communication Technology,"{u'@type': u'Place', u'address': {u'addressReg...",data-scientist,NaN,120000-200000,"Lead Data Scientist, Director Level, Relocate ...",https://www.seek.com.au/job/34423928
22,http://schema.org,JobPosting,Melbourne,2017-09-17T16:59:45Z,"Lead Data Scientist, Director Level, Bangkok\n...",Big Wednesday,Full Time,"{u'@type': u'Organization', u'name': u'Big Wed...",Information & Communication Technology,"{u'@type': u'Place', u'address': {u'addressReg...",data-scientist,NaN,60000-90000,"Lead Data Scientist, Director Level, Relocate ...",https://www.seek.com.au/job/34423928
43,http://schema.org,JobPosting,Melbourne,2017-09-17T16:59:45Z,"Lead Data Scientist, Director Level, Bangkok\n...",Big Wednesday,Full Time,"{u'@type': u'Organization', u'name': u'Big Wed...",Information & Communication Technology,"{u'@type': u'Place', u'address': {u'addressReg...",data-scientist,NaN,90000-120000,"Lead Data Scientist, Director Level, Relocate ...",https://www.seek.com.au/job/34423928
12,http://schema.org,JobPosting,Melbourne,2017-09-11T05:42:25Z,About us\nCrypton Analytics is a high performi...,Crypton Analytics,Full Time,"{u'@type': u'Organization', u'name': u'Crypton...",Science & Technology,"{u'@type': u'Place', u'address': {u'addressLoc...",data-scientist,CBD & Inner Suburbs,0-60000,Data Scientist (Cryptocurrency),https://www.seek.com.au/job/34374593
63,http://schema.org,JobPosting,Melbourne,2017-09-18T14:00:00Z,About us\nCrypton Analytics is a high performi...,Crypton Analytics,Full Time,"{u'@type': u'Organization', u'name': u'Crypton...",Science & Technology,"{u'@type': u'Place', u'address': {u'addressLoc...",data-scientist,CBD & Inner Suburbs,120000-200000,Data Scientist (Cryptocurrency),https://www.seek.com.au/job/34374593
75,http://schema.org,JobPosting,Melbourne,2017-09-11T05:42:25Z,About us\nCrypton Analytics is a high performi...,Crypton Analytics,Full Time,"{u'@type': u'Organization', u'name': u'Crypton...",Science & Technology,"{u'@type': u'Place', u'address': {u'addressLoc...",data-scientist,CBD & Inner Suburbs,120000-200000,Data Scientist (Cryptocurrency),https://www.seek.com.au/job/34374593
21,http://schema.org,JobPosting,Melbourne,2017-09-18T14:00:00Z,About us\nCrypton Analytics is a high performi...,Crypton Analytics,Full Time,"{u'@type': u'Organization', u'name': u'Crypton...",Science & Technology,"{u'@type': u'Place', u'address': {u'addressLoc...",data-scientist,CBD & Inner Suburbs,60000-9

---------------------------------